In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

## Модельная задача

### Аналитическое решение

In [12]:
k = 1. + np.cos(0.5) * np.cos(0.5)
q = 1.
f = np.sin(0.5) * np.sin(0.5)
beta1 = 1.
eps1 = 0
beta2 = 1.
eps2 = 1.

/var/folders/tx/t_y2r90n5gd84735fqzpxzcr0000gn/T/ipykernel_70331/3332729631.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  c1 = ((k*lmb2 + beta2)*(beta1*f - eps1*q) * np.exp(lmb2) + (k*lmb2 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb1 - beta1)*(k*lmb2 + beta2)*np.exp(lmb2) - (k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1)))


In [4]:
def analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2):
    lmb = lambda q,k: np.sqrt(q/k)
    u = lambda c1,c2,lmb,x,f,q: c1 * np.exp(lmb*x) + c2 * np.exp(-lmb*x) + f/q
    lmb1 = lmb(q,k)
    lmb2 = - lmb1
    c1 = ((k*lmb2 + beta2)*(beta1*f - eps1*q) * np.exp(lmb2) + (k*lmb2 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb1 - beta1)*(k*lmb2 + beta2)*np.exp(lmb2) - (k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1)))
    print("c1=", c1)
    c2 = ((k*lmb1 + beta2)*(beta1*f - eps1*q) * np.exp(lmb1) + (k*lmb1 - beta1)*(beta2*f - eps2*q)) / (q * ((k*lmb2 - beta1)*(k*lmb1 + beta2)*np.exp(lmb1) - (k*lmb2 + beta2)*(k*lmb1 - beta1)*np.exp(lmb2)))
    print("c2=", c2)
    u1 = u(c1,c2,lmb1,x,f,q)
    return u1

In [5]:
def analytic_points(k,q,f,beta1,eps1,beta2,eps2):
        x1 = np.linspace(0,1,11)
        y1 = []
        for x in x1:
                res1 = analytic_solution(k,q,f,x,beta1,eps1,beta2,eps2)
                y1.append(res1)
        return y1

### Метод прогонки

In [6]:
def progonkaMod(n,k,q,f,beta1,eps1,beta2,eps2):
    h = 1. / n
    x0 = 1
    solutions = np.zeros(11)
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = k
    bl = -2*k - q*h**2
    cl = k
    dl = -f*h**2
    a0 = k
    b0 = -k-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k-beta2*h
    cL=k
    dL = -eps2*h
    
    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al/(bl+cl*alpha[i-1])
        beta[i] = (dl - cl*beta[i-1])/(bl+cl*alpha[i-1])

    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [7]:
x = np.linspace(0,1,11)
u1 = progonkaMod(10,k,q,f,beta1,eps1,beta2,eps2)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
df = pd.DataFrame({"x": x, "u аналит.": a1, "u числ. ": u1 ,"delta u": delta1})
display(df)
print("delta u max =", max(delta1))

c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387


,x,u аналит.,u числ.,delta u
0,0.0,0.306405,0.311713,0.005309
1,0.1,0.323947,0.329323,0.005376
2,0.2,0.342021,0.347494,0.005473
3,0.3,0.360729,0.366330,0.005601
4,0.4,0.380177,0.385937,0.005761
5,0.5,0.400474,0.406426,0.005952
6,0.6,0.421736,0.427912,0.006177
7,0.7,0.444082,0.450517,0.006436
8,0.8,0.467639,0.474369,0.006730
9,0.9,0.492540,0.499602,0.007063


delta u max = 0.007433906101123489


## Задача с переменными коеффициентами

In [8]:
def progonkaVar(n,beta1,eps1,beta2,eps2):
    h = 1/n
    x1 = np.linspace(0,1,11); x0 = 1; x = 0 + h
    solutions = np.zeros(11)
    k = lambda x: 1 + np.cos(x) * np.cos(x)
    q = lambda x: 1
    f = lambda x: np.sin(x) * np.sin(x)
    
    u = np.zeros(n+1)
    alpha = np.zeros(n+1)
    beta = np.zeros(n+1)
    al = lambda x: k(x+h/2)
    bl = lambda x: -(k(x+h/2)+k(x-h/2)+q(x)*h**2)
    cl = lambda x: k(x-h/2)
    dl = lambda x: -f(x)*h**2
    a0 = k(0)
    b0 = -k(0)-beta1*h
    c0 = 0
    d0 = -eps1*h
    aL = 0
    bL = -k(1)-beta2*h
    cL = k(1)
    dL = -eps2*h

    alpha[0] = -a0/b0; beta[0] = d0/b0
    for i in range(1,n+1):
        alpha[i] = -al(x)/(bl(x)+cl(x)*alpha[i-1])
        beta[i] = (dl(x) - cl(x)*beta[i-1])/(bl(x)+cl(x)*alpha[i-1])
        x += h
    u[-1] = (dL-cL*beta[-2])/(bL+cL*alpha[-2]); 
    solutions[-1] = u[-1]
    count = 9
    t = 0.9
    for i in range(n-1,-1,-1):
        x0 -= h
        u[i] = alpha[i]*u[i+1]+beta[i]
        if abs(x0 - t) < 10**(-10):
            solutions[count] = u[i]
            count -= 1
            t -= 0.1
    
    return solutions

In [9]:
x = np.linspace(0,1,11)
u2 = progonkaVar(1000,beta1,eps1,beta2,eps2)
a2 = progonkaVar(10**6,beta1,eps1,beta2,eps2)
delta2 = []
for i in range(len(u2)):
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u аналит.": a2, "u числ. ": u2 ,"delta u ": delta2})
display(df)
print("delta u max =", max(delta2))

,x,u аналит.,u числ.,delta u
0,0.0,0.305154,0.305252,0.000097
1,0.1,0.321211,0.321306,0.000095
2,0.2,0.338997,0.339090,0.000093
3,0.3,0.358680,0.358771,0.000091
4,0.4,0.380389,0.380478,0.000089
5,0.5,0.404216,0.404303,0.000087
6,0.6,0.430207,0.430293,0.000086
7,0.7,0.458356,0.458441,0.000085
8,0.8,0.488587,0.488672,0.000084
9,0.9,0.520736,0.520820,0.000084


delta u max = 9.749360475752589e-05


## Таблица результатов

In [10]:
x = np.linspace(0,1,11)
a1 = analytic_points(k,q,f,beta1,eps1,beta2,eps2)
a2 = progonkaVar(10**6,beta1,eps1,beta2,eps2)

c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387
c1= 0.15342694478418903
c2= -0.07687109521568387


In [11]:
delta1 = []
delta2 = []
u1 = progonkaMod(10,k,q,f,beta1,eps1,beta2,eps2)
u2 = progonkaVar(10,beta1,eps1,beta2,eps2)
for i in range(len(u1)):
    delta1.append(abs(u1[i] - a1[i]))
    delta2.append(abs(u2[i] - a2[i]))
df = pd.DataFrame({"x": x, "u мод. аналит.": a1, "u мод. числ. ": u1 ,"du мод.": delta1, 
                  "u пер. аналит.": a2, "u пер. числ. ": u2 ,"du": delta2})
display(df)
print("du мод. макс =", max(delta1))
print("du перем. макс = ", max(delta2))

,x,u мод. аналит.,u мод. числ.,du мод.,u пер. аналит.,u пер. числ.,du
0,0.0,0.306405,0.311713,0.005309,0.305154,0.314362,0.009208
1,0.1,0.323947,0.329323,0.005376,0.321211,0.330080,0.008869
2,0.2,0.342021,0.347494,0.005473,0.338997,0.347574,0.008577
3,0.3,0.360729,0.366330,0.005601,0.358680,0.367009,0.008328
4,0.4,0.380177,0.385937,0.005761,0.380389,0.388511,0.008121
5,0.5,0.400474,0.406426,0.005952,0.404216,0.412171,0.007956
6,0.6,0.421736,0.427912,0.006177,0.430207,0.438039,0.007832
7,0.7,0.444082,0.450517,0.006436,0.458356,0.466109,0.007753
8,0.8,0.467639,0.474369,0.006730,0.488587,0.496311,0.007723
9,0.9,0.492540,0.499602,0.007063,0.520736,0.528485,0.007749


du мод. макс = 0.007433906101123489
du перем. макс =  0.009207842238538044
